In [1]:
import numpy as np
import tensorflow as tf
from numpy.random import rand, randint, exponential

from src.environment.network_rm import network_rm
from src.agent.dqn.dqn import exp_replay
from src.agent.dqn.models  import MLP

In [2]:
inventory = np.array([17,21,5,100]) # the last item is time
demand_types = np.array([[1,0,0,0], [0,2,1,0], [0,0,3,0]])
demand_values = np.array([2,7,1])
demand_arrivals = np.array([0.1, 0.5, 0.4])
inter_arrival_time = 1
g = network_rm(inventory, demand_values, demand_types, demand_arrivals, inter_arrival_time)

In [3]:
tf.reset_default_graph()
session = tf.InteractiveSession()

# Brain maps from observation to Q values for different actions.
# Here it is a done using a multi layer perceptron with 2 hidden
# layers

#observation_shape has to be a 1d vector for now
brain = MLP(list(g.observation_shape), [200, 200, g.num_actions], # change size to larger observation arrays
            [tf.tanh, tf.tanh, tf.identity])

# The optimizer to use. Here we use RMSProp as recommended
# by the publication
optimizer = tf.train.RMSPropOptimizer(learning_rate= 0.001, decay=0.9)

# DiscreteDeepQ object
current_controller = exp_replay(g.observation_shape, g.num_actions, brain, optimizer, session,
                                   discount_rate=0.99, exploration_period=5000, max_experience=10000, 
                                   store_every_nth=4, train_every_nth=4)

session.run(tf.initialize_all_variables())
session.run(current_controller.target_network_update)

Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


...

Instructions for updating:
Please switch to tf.summary.merge.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [46]:
def run_sim(env, agent, steps = 100, disable_training = False):
    last_observation = None
    last_action      = None
    
    for s in range(steps):
        if env.terminate:
            break
        new_observation = env.observe()
        reward          = env.collect_reward()

        # store last transition
        if last_observation is not None:
            agent.store(last_observation, last_action, reward, new_observation)

        # act
        new_action = agent.action(new_observation)
        env.perform_action(new_action)
    
        #transition
        env.transition()
        
        #train
        if not disable_training:
            agent.training_step()

        # update current state as last state.
        last_action = new_action
        last_observation = new_observation
    return s


## One instance of network rm problem

In [53]:
g = network_rm(inventory, demand_values, demand_types, demand_arrivals, inter_arrival_time)

In [54]:
g.state

array([  17.,   21.,    5.,  100.,    0.,    0.,    0.,    0.])

In [58]:
x = g.total_reward
print(x)

67


In [56]:
T = 100
T = run_sim(g, current_controller, T)
print("average reward: {}".format((g.total_reward - x)/T))

average reward: 0.6767676767676768


In [57]:
g.state

array([  0.        ,  11.        ,   0.        ,   2.75622577,
         0.        ,   2.        ,   1.        ,   0.        ])

16